In [1]:
from retrain import *
import tensorflow as tf
import numpy as np
import time

In [2]:
#test on google's image laerning using imagenet input file structure
IMGDIR = "/Users/zhouyu/Documents/Zhou_Yu/DS/Galvanize/Capstone_data/test_data_stan"
class Helper_FLAGS(object):
    def __init__(self):
        self.model_dir = "./temp"
        self.image_dir = IMGDIR
        self.testing_percentage = 10
        self.validation_percentage = 10
        self.flip_left_right = False
        self.random_crop = False
        self.random_scale = False
        self.random_brightness = False
        self.bottleneck_dir = './temp/bottleneck'
        self.architecture = 'inception_v3'
        self.final_tensor_name = 'final_result'
        self.summaries_dir = './temp/retrain_logs'
        self.how_many_training_steps = 10
        self.train_batch_size = 32
        self.eval_step_interval = 10
        self.validation_batch_size = 32
        self.intermediate_store_frequency = 0
        self.intermediate_output_graphs_dir = "./temp/intermediate_graph"
        self.print_misclassified_test_images = False
        self.output_graph = './temp/output_graph.pb'
        self.output_labels = './temp/output_labels.txt'
        self.test_batch_size = -1
zFLAGS = Helper_FLAGS()

In [3]:
def getNumjpegs(lst):
    count = 0
    for cat, vals in lst.iteritems():
        count += len(vals['testing'])+len(vals['training'])+len(vals['validation'])
    return count  

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)
model_info = create_model_info('inception_v3')

In [5]:
model_info

{'bottleneck_tensor_name': 'pool_3/_reshape:0',
 'bottleneck_tensor_size': 2048,
 'data_url': 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz',
 'input_depth': 3,
 'input_height': 299,
 'input_mean': 128,
 'input_std': 128,
 'input_width': 299,
 'model_file_name': 'classify_image_graph_def.pb',
 'resized_input_tensor_name': 'Mul:0'}

In [6]:
  # Set up the pre-trained graph.
maybe_download_and_extract(model_info['data_url'])
graph, bottleneck_tensor, resized_image_tensor = (create_model_graph(model_info))

In [7]:
# Look at the folder structure, and create lists of all the images.
image_lists = create_image_lists(zFLAGS.image_dir, zFLAGS.testing_percentage,
                                   zFLAGS.validation_percentage)
class_count = len(image_lists.keys())
assert class_count>1

INFO:tensorflow:Looking for images in 'Chihuahua'
INFO:tensorflow:Looking for images in 'Shih-Tzu'


In [8]:
 # See if the command-line flags mean we're applying any distortions.
do_distort_images = should_distort_images(zFLAGS.flip_left_right, zFLAGS.random_crop, 
                                          zFLAGS.random_scale,zFLAGS.random_brightness)

In [9]:
start_time = time.time()
with tf.Session(graph=graph) as sess:
    # Set up the image decoding sub-graph.
    jpeg_data_tensor, decoded_image_tensor = add_jpeg_decoding(
        model_info['input_width'], model_info['input_height'],
        model_info['input_depth'], model_info['input_mean'],
        model_info['input_std'])

    if do_distort_images:
        # We will be applying distortions, so setup the operations we'll need.
        (distorted_jpeg_data_tensor,
         distorted_image_tensor) = add_input_distortions(
           zFLAGS.flip_left_right, zFLAGS.random_crop, zFLAGS.random_scale,
           zFLAGS.random_brightness, model_info['input_width'],
           model_info['input_height'], model_info['input_depth'],
           model_info['input_mean'], model_info['input_std'])
    else:
    # We'll make sure we've calculated the 'bottleneck' image summaries and
      # cached them on disk.
        cache_bottlenecks(sess, image_lists, zFLAGS.image_dir,
                        zFLAGS.bottleneck_dir, jpeg_data_tensor,
                        decoded_image_tensor, resized_image_tensor,
                        bottleneck_tensor, zFLAGS.architecture)

    # Add the new layer that we'll be training.
    (train_step, cross_entropy, bottleneck_input, ground_truth_input,
     final_tensor) = add_final_training_ops(
         len(image_lists.keys()), zFLAGS.final_tensor_name, bottleneck_tensor,
         model_info['bottleneck_tensor_size'])

    # Create the operations we need to evaluate the accuracy of our new layer.
    evaluation_step, prediction = add_evaluation_step(
        final_tensor, ground_truth_input)

    # Merge all the summaries and write them out to the summaries_dir
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(zFLAGS.summaries_dir + '/train',
                                         sess.graph)

    validation_writer = tf.summary.FileWriter(
        zFLAGS.summaries_dir + '/validation')

    # Set up all our weights to their initial default values.
    init = tf.global_variables_initializer()
    sess.run(init)

    # Run the training for as many cycles as requested on the command line.
    for i in range(zFLAGS.how_many_training_steps):
    # Get a batch of input bottleneck values, either calculated fresh every
      # time with distortions applied, or from the cache stored on disk.
        if do_distort_images:
            (train_bottlenecks,
             train_ground_truth) = get_random_distorted_bottlenecks(
             sess, image_lists, zFLAGS.train_batch_size, 'training',
             zFLAGS.image_dir, distorted_jpeg_data_tensor,
             distorted_image_tensor, resized_image_tensor, bottleneck_tensor)
        else:
            (train_bottlenecks,
             train_ground_truth, _) = get_random_cached_bottlenecks(
             sess, image_lists, zFLAGS.train_batch_size, 'training',
             zFLAGS.bottleneck_dir, zFLAGS.image_dir, jpeg_data_tensor,
             decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
             zFLAGS.architecture)
    # Feed the bottlenecks and ground truth into the graph, and run a training
    # step. Capture training summaries for TensorBoard with the `merged` op.
        train_summary, _ = sess.run([merged, train_step],
                                    feed_dict={bottleneck_input: train_bottlenecks,
                                               ground_truth_input: train_ground_truth})
        train_writer.add_summary(train_summary, i)

    # Every so often, print out how well the graph is training.
        is_last_step = (i + 1 == zFLAGS.how_many_training_steps)
        if (i % zFLAGS.eval_step_interval) == 0 or is_last_step:
            train_accuracy, cross_entropy_value = sess.run(
                [evaluation_step, cross_entropy],
                feed_dict={bottleneck_input: train_bottlenecks,
                       ground_truth_input: train_ground_truth})
            tf.logging.info('%s: Step %d: Train accuracy = %.1f%%' %
                        (datetime.now(), i, train_accuracy * 100))
            tf.logging.info('%s: Step %d: Cross entropy = %f' %
                        (datetime.now(), i, cross_entropy_value))
            validation_bottlenecks, validation_ground_truth, _ = (
                get_random_cached_bottlenecks(
                    sess, image_lists, zFLAGS.validation_batch_size, 'validation',
                    zFLAGS.bottleneck_dir, zFLAGS.image_dir, jpeg_data_tensor,
                    decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
                    zFLAGS.architecture))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
            validation_summary, validation_accuracy = sess.run(
                [merged, evaluation_step],
                feed_dict={bottleneck_input: validation_bottlenecks,
                           ground_truth_input: validation_ground_truth})
            validation_writer.add_summary(validation_summary, i)
            tf.logging.info('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
                            (datetime.now(), i, validation_accuracy * 100,
                             len(validation_bottlenecks)))

        # Store intermediate results
        intermediate_frequency = zFLAGS.intermediate_store_frequency

        if (intermediate_frequency > 0 and (i % intermediate_frequency == 0)
            and i > 0):
            intermediate_file_name = (zFLAGS.intermediate_output_graphs_dir +
                                  'intermediate_' + str(i) + '.pb')
            tf.logging.info('Save intermediate result to : ' +
                        intermediate_file_name)
            save_graph_to_file(sess, graph, intermediate_file_name)
    end_time = time.time()
    print "training done, now testing...."
    # We've completed all our training, so run a final test evaluation on
    # some new images we haven't used before.
    test_bottlenecks, test_ground_truth, test_filenames = (
        get_random_cached_bottlenecks(
            sess, image_lists, zFLAGS.test_batch_size, 'testing',
            zFLAGS.bottleneck_dir, zFLAGS.image_dir, jpeg_data_tensor,
            decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
            zFLAGS.architecture))
    test_accuracy, predictions = sess.run(
        [evaluation_step, prediction],
        feed_dict={bottleneck_input: test_bottlenecks,
                   ground_truth_input: test_ground_truth})
    tf.logging.info('Final test accuracy = %.1f%% (N=%d)' %
                    (test_accuracy * 100, len(test_bottlenecks)))

    if zFLAGS.print_misclassified_test_images:
        tf.logging.info('=== MISCLASSIFIED TEST IMAGES ===')
        for i, test_filename in enumerate(test_filenames):
            if predictions[i] != test_ground_truth[i].argmax():
                tf.logging.info('%70s  %s' %
                          (test_filename,
                           list(image_lists.keys())[predictions[i]]))

    # Write out the trained graph and labels with the weights stored as
    # constants.
    save_graph_to_file(sess, graph, zFLAGS.output_graph)
    with gfile.FastGFile(zFLAGS.output_labels, 'w') as f:
        f.write('\n'.join(image_lists.keys()) + '\n')
    saver = tf.train.Saver()
    saver.save(sess, "my_model")
print "total 10 training for {} images is {} s".format(getNumjpegs(image_lists), end_time - start_time)

INFO:tensorflow:2017-09-01 11:50:06.511387: Step 0: Train accuracy = 96.9%
INFO:tensorflow:2017-09-01 11:50:06.514854: Step 0: Cross entropy = 0.532486
INFO:tensorflow:2017-09-01 11:50:06.580539: Step 0: Validation accuracy = 100.0% (N=32)
INFO:tensorflow:2017-09-01 11:50:06.899064: Step 9: Train accuracy = 100.0%
INFO:tensorflow:2017-09-01 11:50:06.900300: Step 9: Cross entropy = 0.165419
INFO:tensorflow:2017-09-01 11:50:06.959527: Step 9: Validation accuracy = 100.0% (N=32)
training done, now testing....
INFO:tensorflow:Final test accuracy = 100.0% (N=1)
INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.
total 10 training for 44 images is 5.98612499237 s
